In [ ]:
# Importing libraries/functions the project needs
from urllib.parse import quote_plus
from string import ascii_uppercase
import os
import re
import pandas as pd
import sqlite3

In [ ]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [ ]:
df = pd.read_excel(spreadsheet_url)
df.head()

,第15任總統副總統選舉候選人在臺北市各投開票所得票數一覽表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,鄉(鎮、市、區)別,村里別,投票所別,各組候選人得票情形,NaN,NaN,有效票數A\nA=1+2+...+N,無效票數B,投票數C\nC=A+B,已領未投票數\nD\nD=E-C,發出票數E\nE=C+D,用餘票數F,選舉人數G\nG=E+F,投票率H\nH=C÷G
1,NaN,NaN,NaN,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,總 計,NaN,NaN,"70,769","685,830","875,854","1,632,453","21,381","1,653,834",143,"1,653,977","513,287","2,167,264",76.3098


In [ ]:
file_name = "區域立委-A05-2-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市第1選舉區', '臺北市第2選舉區', '臺北市第3選舉區', '臺北市第4選舉區', '臺北市第5選舉區', '臺北市第6選舉區', '臺北市第7選舉區', '臺北市第8選舉區']


In [ ]:
regional_legislator_taipei_city = {}
for sheet_name in xl.sheet_names:
    regional_legislator_taipei_city[sheet_name] = pd.read_excel(spreadsheet_url, sheet_name=sheet_name)
for k, v in regional_legislator_taipei_city.items():
    print("Shape of {} : {}".format(k, v.shape))

Shape of 臺北市第1選舉區 : (219, 18)
Shape of 臺北市第2選舉區 : (224, 17)
Shape of 臺北市第3選舉區 : (233, 15)
Shape of 臺北市第4選舉區 : (248, 18)
Shape of 臺北市第5選舉區 : (225, 19)
Shape of 臺北市第6選舉區 : (200, 21)
Shape of 臺北市第7選舉區 : (222, 16)
Shape of 臺北市第8選舉區 : (212, 20)


In [ ]:
file_name = "不分區立委-A05-6-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [ ]:
file_name = "山地立委-A05-4-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [ ]:
file_name = "平地立委-A05-4-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [ ]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')

In [ ]:
n_cols = df.columns.size
n_candidates = n_cols - 11
id_vars = ['district', 'village', 'office']
candidates = list(df.columns[3:(3 + n_candidates)])
office_cols = list(ascii_uppercase[:8])
col_names = id_vars + candidates + office_cols
df.columns = col_names

In [ ]:
filled_district = df['district'].fillna(method='ffill')
df = df.assign(district=filled_district)
df.head()

,district,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
0,總 計,NaN,NaN,70769,685830,875854,1632453,21381,1653834,143,1653977,513287,2167264,76.3098
1,松山區,NaN,NaN,5436,55918,64207,125561,1762,127323,2,127325,37329,164654,77.3276
2,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811


In [ ]:
df = df.dropna()

In [ ]:
print(df['district'].unique())
stripped_strict = df['district'].str.replace("\u3000", "")
df = df.assign(district=stripped_strict)
print(df['district'].unique())

['\u3000松山區' '\u3000信義區' '\u3000大安區' '\u3000中山區' '\u3000中正區' '\u3000大同區'
 '\u3000萬華區' '\u3000文山區' '\u3000南港區' '\u3000內湖區' '\u3000士林區' '\u3000北投區']
['松山區' '信義區' '大安區' '中山區' '中正區' '大同區' '萬華區' '文山區' '南港區' '內湖區' '士林區' '北投區']


In [ ]:
df = df.drop(labels=office_cols, axis=1)
df_long = pd.melt(df,
                  id_vars=id_vars,
                  var_name='candidate_info',
                  value_name='votes'
                 )
df_long.head()

,district,village,office,candidate_info,votes
0,松山區,莊敬里,573.0,(1)\n宋楚瑜\n余湘,36
1,松山區,莊敬里,574.0,(1)\n宋楚瑜\n余湘,46
2,松山區,莊敬里,575.0,(1)\n宋楚瑜\n余湘,48
3,松山區,莊敬里,576.0,(1)\n宋楚瑜\n余湘,43
4,松山區,東榮里,577.0,(1)\n宋楚瑜\n余湘,38


In [ ]:
def tidy_dataframe(df):
    # updating columns attributes 
    n_cols = df.columns.size
    n_candidates = n_cols - 11
    id_vars = ['district', 'village', 'office']
    candidates = list(df.columns[3:(3 + n_candidates)])
    office_cols = list(ascii_uppercase[:8])
    col_names = id_vars + candidates + office_cols
    df.columns = col_names
    # forward-fill district values
    filled_district = df['district'].fillna(method='ffill')
    df = df.assign(district=filled_district)
    # removing summations
    df = df.dropna()
    # removing extra spaces
    stripped_strict = df['district'].str.replace("\u3000", "")
    df = df.assign(district=stripped_strict)
    # pivoting
    df = df.drop(labels=office_cols, axis=1)
    tidy_df = pd.melt(df,
                      id_vars=id_vars,
                      var_name='candidate_info',
                      value_name='votes'
                     )
    return tidy_df

In [ ]:
presidential = pd.DataFrame()
city_county_list = ['宜蘭縣', '彰化縣', '金門縣', '桃園市', '苗栗縣', '臺南市', '雲林縣', '南投縣', '高雄市', '臺北市', '新北市', '花蓮縣', '新竹市', '新竹縣', '基隆市', '連江縣', '嘉義縣', '嘉義市', '屏東縣', '澎湖縣', '臺東縣', '臺中市']
for city_county in city_county_list:
    file_name = "總統-A05-4-候選人得票數一覽表-各投開票所({}).xls".format(city_county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['city_county'] = city_county
    presidential = presidential.append(tidy_df)
    print("Tidying {}".format(file_name))
presidential = presidential.reset_index(drop=True) # reset index for the appended dataframe

Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(宜蘭縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(彰化縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(金門縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(桃園市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(苗栗縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺南市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(雲林縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(南投縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(高雄市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(花蓮縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(基隆市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(連江縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(屏東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(澎湖縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺中市).xls


In [ ]:
presidential.head()

,district,village,office,candidate_info,votes,city_county
0,宜蘭市,民族里,1.0,(1)\n宋楚瑜\n余湘,37,宜蘭縣
1,宜蘭市,民族里,2.0,(1)\n宋楚瑜\n余湘,31,宜蘭縣
2,宜蘭市,建軍里,3.0,(1)\n宋楚瑜\n余湘,19,宜蘭縣
3,宜蘭市,建軍里,4.0,(1)\n宋楚瑜\n余湘,29,宜蘭縣
4,宜蘭市,泰山里,5.0,(1)\n宋楚瑜\n余湘,25,宜蘭縣


In [ ]:
def adjust_presidential(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "")
    candidates = candidate_info_df[1].str.cat(candidate_info_df[2], sep="/")
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['office'] = df['office'].astype(int)
    return df

In [ ]:
presidential = adjust_presidential(presidential)
presidential = presidential[['city_county', 'district', 'village', 'office', 'number', 'candidate', 'votes']]

In [ ]:
regional = pd.DataFrame()
for city_county in city_county_list:
    file_name = "區域立委-A05-2-得票數一覽表({}).xls".format(city_county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
    xl = pd.ExcelFile(spreadsheet_url)
    for sheet in xl.sheet_names:
        # skip those combined cells
        df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',', sheet_name=sheet)
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['city_county'] = city_county
        tidy_df['electoral_district'] = sheet
        regional = regional.append(tidy_df)
        print("Tidying {} of {}".format(sheet, file_name))
regional = regional.reset_index(drop=True) # reset index for the appended dataframe

Tidying 宜蘭縣選舉區 of 區域立委-A05-2-得票數一覽表(宜蘭縣).xls
Tidying 彰化縣第1選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第2選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第3選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第4選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 金門縣選舉區 of 區域立委-A05-2-得票數一覽表(金門縣).xls
Tidying 桃園市第1選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第2選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第3選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第4選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第5選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第6選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 苗栗縣第1選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 苗栗縣第2選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 臺南市第1選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第2選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第3選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第4選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第5選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第6選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 雲林縣第1選舉區 of 區域立委-A05-2-得票數一覽表(雲林縣).xls
Tidying 雲林縣第2選舉區 

In [ ]:
def adjust_legislative(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "")
    candidates = candidate_info_df[1]
    parties = candidate_info_df[2]
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['party'] = parties
    df['office'] = df['office'].astype(int)
    return df

In [ ]:
regional = adjust_legislative(regional)
regional = regional[['city_county', 'electoral_district', 'district', 'village', 'office', 'number', 'party', 'candidate', 'votes']]

In [ ]:
indigenous = pd.DataFrame()
indigenous_types = ['山地', '平地']
for city_county in city_county_list:
    for indigenous_type in indigenous_types:
        file_name = "{}立委-A05-4-得票數一覽表({}).xls".format(indigenous_type, city_county)
        file_url = quote_plus(file_name)
        spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
        # skip those combined cells
        df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['city_county'] = city_county
        tidy_df['electoral_district'] = '{}原住民'.format(indigenous_type)
        indigenous = indigenous.append(tidy_df)
        print("Tidying {}".format(file_name))
indigenous = indigenous.reset_index(drop=True) # reset index for the appended dataframe

Tidying 山地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 平地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 山地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 平地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 山地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 平地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 山地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 平地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 山地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 平地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 山地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 山地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 平地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 山地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 平地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 山地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 平地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 山地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 山地立委-A05-4-得票數一覽表(新北市).xls
Tidying 平地立委-A05-4-得票數一覽表(新北市).xls
Tidying 山地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 平地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 山地立委-A05-4-得

In [ ]:
indigenous = adjust_legislative(indigenous)
indigenous = indigenous[['city_county', 'electoral_district', 'district', 'village', 'office', 'number', 'party', 'candidate', 'votes']]
indigenous.head()

,city_county,electoral_district,district,village,office,number,party,candidate,votes
0,宜蘭縣,山地原住民,宜蘭市,民族里,1,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
1,宜蘭縣,山地原住民,宜蘭市,民族里,2,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
2,宜蘭縣,山地原住民,宜蘭市,建軍里,3,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
3,宜蘭縣,山地原住民,宜蘭市,建軍里,4,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
4,宜蘭縣,山地原住民,宜蘭市,泰山里,5,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0


In [ ]:
legislative_regional = pd.concat([regional, indigenous], axis=0)
legislative_regional = legislative_regional.reset_index(drop=True)

In [ ]:
legislative_at_large = pd.DataFrame()
for city_county in city_county_list:
    file_name = "不分區立委-A05-6-得票數一覽表({}).xls".format(city_county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['city_county'] = city_county
    legislative_at_large = legislative_at_large.append(tidy_df)
    print("Tidying {}".format(file_name))
legislative_at_large = legislative_at_large.reset_index(drop=True) # reset index for the appended dataframe

Tidying 不分區立委-A05-6-得票數一覽表(宜蘭縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(彰化縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(金門縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(桃園市).xls
Tidying 不分區立委-A05-6-得票數一覽表(苗栗縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺南市).xls
Tidying 不分區立委-A05-6-得票數一覽表(雲林縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(南投縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(高雄市).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(花蓮縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(基隆市).xls
Tidying 不分區立委-A05-6-得票數一覽表(連江縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義市).xls
Tidying 不分區立委-A05-6-得票數一覽表(屏東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(澎湖縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺中市).xls


In [ ]:
legislative_at_large = adjust_legislative(legislative_at_large)
legislative_at_large = legislative_at_large[['city_county', 'district', 'village', 'office', 'number', 'party', 'votes']]
legislative_at_large.head()

,city_county,district,village,office,number,party,votes
0,宜蘭縣,宜蘭市,民族里,1,1,合一行動聯盟,2
1,宜蘭縣,宜蘭市,民族里,2,1,合一行動聯盟,1
2,宜蘭縣,宜蘭市,建軍里,3,1,合一行動聯盟,2
3,宜蘭縣,宜蘭市,建軍里,4,1,合一行動聯盟,0
4,宜蘭縣,宜蘭市,泰山里,5,1,合一行動聯盟,3


In [ ]:
presidential.to_csv('presidential.csv', index=False)
legislative_regional.to_csv('legislative_regional.csv', index=False)
legislative_at_large.to_csv('legislative_at_large.csv', index=False)

In [ ]:
conn = sqlite3.connect('tw-election-2020.db')
presidential.to_sql('presidential', con=conn, index=False, if_exists='replace')
legislative_regional.to_sql('legislative_regional', con=conn, index=False, if_exists='replace')
legislative_at_large.to_sql('legislative_at_large', con=conn, index=False, if_exists='replace')

In [ ]:
ttl_votes_by_candidates = presidential.groupby(['city_county', 'district', 'village', 'candidate'])['votes'].sum()
ttl_votes_by_villages = presidential.groupby(['city_county', 'district', 'village'])['votes'].sum()
ttl_votes_by_candidates = pd.DataFrame(ttl_votes_by_candidates).reset_index()
ttl_votes_by_villages = pd.DataFrame(ttl_votes_by_villages).reset_index()
presidential_votes_summary = pd.merge(ttl_votes_by_candidates, ttl_votes_by_villages,
                                      left_on=['city_county', 'district', 'village'],
                                      right_on=['city_county', 'district', 'village'])
vote_ratios = presidential_votes_summary['votes_x'] / presidential_votes_summary['votes_y']
presidential_votes_summary['votes_ratio'] = vote_ratios
presidential_votes_summary.head()

,city_county,district,village,candidate,votes_x,votes_y,votes_ratio
0,南投縣,中寮鄉,中寮村,宋楚瑜/余湘,18,443,0.040632
1,南投縣,中寮鄉,中寮村,蔡英文/賴清德,208,443,0.469526
2,南投縣,中寮鄉,中寮村,韓國瑜/張善政,217,443,0.489842
3,南投縣,中寮鄉,內城村,宋楚瑜/余湘,17,297,0.057239
4,南投縣,中寮鄉,內城村,蔡英文/賴清德,139,297,0.468013


In [ ]:
presidential_votes_summary[presidential_votes_summary['candidate'] == '韓國瑜/張善政'].to_csv('summary_kmt.csv', index=False)
presidential_votes_summary[presidential_votes_summary['candidate'] == '蔡英文/賴清德'].to_csv('summary_dpp.csv', index=False)